In [26]:
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

import torch.nn as nn
import numpy as np



window_size = 50  # size of the window


data_path = "../Anomaly/DCDetector_dataset/SMD/"
input_dim = 38

scaler = StandardScaler()
data = np.load(data_path + "/SMD_train.npy")[:,:]
scaler.fit(data)
data = scaler.transform(data)
test_data = np.load(data_path + "/SMD_test.npy")[:,:]
test = scaler.transform(test_data)
train_data = data
data_len = len(train_data)
val_data = train_data[(int)(data_len * 0.8):]
test_labels = np.load(data_path + "/SMD_test_label.npy")[:]


# data_path = "../Anomaly/DCDetector_dataset/SWAT/"
# input_dim = 51
# train_data = pd.read_csv( data_path + 'swat_train2.csv')
# test_data = pd.read_csv(data_path + 'swat2.csv')

# test_labels = test_data.values[:, -1]
# train_data = train_data.values[:, :-1]
# test_data = test_data.values[:, :-1]

# scaler = StandardScaler()
# scaler.fit(train_data)

# train_data = scaler.transform(train_data)
# test_data = scaler.transform(test_data)
# data_len = len(train_data)
# val_data = train_data[(int)(data_len * 0.8):]


# data_path = "../Anomaly/DCDetector_dataset/MSL/"
# input_dim = 55

# scaler = StandardScaler()
# train_data = np.load(data_path + "/MSL_train.npy")
# scaler.fit(train_data)
# train_data = scaler.transform(train_data)
# test_data = np.load(data_path + "/MSL_test.npy")
# test_data = scaler.transform(test_data)
# test_labels = np.load(data_path + "/MSL_test_label.npy")
# # val_data = test_data
# data_len = len(train_data)
# val_data = train_data[(int)(data_len * 0.8):]


print("# of train: ", len(train_data))
print("# of test: ", len(test_data))


df_test_0 = test_labels[test_labels == 0]
df_test_1 = test_labels[test_labels == 1]

print("number of 1s in test: ", len(df_test_1))
print("number of 0s in test: ", len(df_test_0))

# of train:  708405
# of test:  708420
number of 1s in test:  29444
number of 0s in test:  678976


In [27]:
window_size = 50  # size of the window

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class AttentionLayer(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(AttentionLayer, self).__init__()
        self.attention = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, 1),
            nn.Softmax(dim=1)
        )

    def forward(self, inputs):
        attention_weights = self.attention(inputs)
        weighted_input = inputs * attention_weights
        return weighted_input, attention_weights

class MSEFeedbackRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(MSEFeedbackRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        batch_size = x.size(0)
        x = x.unsqueeze(1)
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out)  # Apply the fully connected layer to each time step
        return out


class CompositeModel(nn.Module):
    def __init__(self, autoencoder, rnn, attention_dim=64):
        super(CompositeModel, self).__init__()
        self.autoencoder = autoencoder
        self.rnn = rnn
        self.attention = AttentionLayer(input_dim= input_dim+1, hidden_dim=attention_dim)

    def forward(self, x, y):
        # reconstructed, mean, log = self.autoencoder(x) # For VAE
        reconstructed = self.autoencoder(x) #For AE
        mse_error = ((y - reconstructed) ** 2).mean(dim=1, keepdim=True)

        combined_input = torch.cat((reconstructed, mse_error), dim=1)
        combined_input, attention_weights = self.attention(combined_input)

        rnn_output = self.rnn(combined_input)
        rnn_output = rnn_output.squeeze(1)
        
        adjusted_reconstructed = reconstructed + rnn_output
        return adjusted_reconstructed, mse_error, rnn_output


if "SWAT" in data_path:
    MODEL_NAME = "models/SWAT/SWAT-AE-FAR"
if "MSL" in data_path:
    MODEL_NAME = "models/MSL/MSL-AE-FAR"
if "SMD" in data_path:
    MODEL_NAME = "models/SMD/SMD-AE-FAR"

model = torch.load(MODEL_NAME + ".pt", map_location=device)


model.to(device)
batch_size = 128  # size of each batch


# to predict a single target value, not the entire window
def iterate_batches(data, window_size, batch_size, start_idx = 0):
    for start in range(start_idx, len(data) - window_size, batch_size):
        end = min(start + batch_size, len(data) - window_size)
        batch_data = [data[i:i + window_size] for i in range(start, end)]
        batch_targets = [data[i + window_size] for i in range(start, end)]
        yield torch.stack(batch_data), torch.stack(batch_targets)


from sklearn.metrics import precision_recall_fscore_support
import torch
import numpy as np

X_test = torch.tensor(test_data, dtype=torch.float32).to(device)

predictions = None
for batch, y_batch in iterate_batches(X_test, window_size, batch_size):
    # y_pred = model(batch)
    y_pred, mean, log_var = model(batch, y_batch)
    
    y_pred = y_pred.cpu().detach().numpy()
    
    if predictions is None:
        predictions = y_pred
    else:
        predictions = np.concatenate((predictions, y_pred), axis=0)

print("X_test ", len(X_test))
print("predictions ", len(predictions))

/tmp/ipykernel_323023/1114789134.py:67: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(MODEL_NAME + ".pt", map_location=device)


X_test  708420
predictions  708370


In [28]:
def apply_adjustment(gt_, pred_):
    gt = gt_.copy()
    pred = pred_.copy()
    anomaly_state = False
    for i in range(len(gt)):
        if gt[i] == 1 and pred[i] == 1 and not anomaly_state:
            anomaly_state = True
            for j in range(i, 0, -1):
                if gt[j] == 0:
                    break
                else:
                    if pred[j] == 0:
                        pred[j] = 1
            for j in range(i, len(gt)):
                if gt[j] == 0:
                    break
                else:
                    if pred[j] == 0:
                        pred[j] = 1
        elif gt[i] == 0:
            anomaly_state = False
        if anomaly_state:
            pred[i] = 1
    return gt, pred
# end function

def sliding_window_anomaly_detection(mse_list, window_size, threshold_factor=3):
    mse_series = pd.Series(mse_list)
    
    # Calculate moving average and moving standard deviation
    moving_avg = mse_series.rolling(window=window_size, min_periods=1).mean()
    moving_std = mse_series.rolling(window=window_size, min_periods=1).std()
    
    # Calculate dynamic threshold
    dynamic_threshold = moving_avg + (threshold_factor * moving_std)
    
    # Identify anomalies
    anomalies = (mse_series > dynamic_threshold).astype(int)
    10
    # Convert to list for output
    anomalies_list = anomalies.tolist()
    
    return anomalies_list, dynamic_threshold.tolist()

def get_precision_recall_f1(true_labels, pred_y):
    precision, recall, f1_score, _ = precision_recall_fscore_support(true_labels, pred_y, average='binary')
    return round(precision, 4), round(recall, 4), round(f1_score, 4)

In [30]:

if "SWAT" in MODEL_NAME:
    threshold_fixed = 7 #SWAT
    th_factor = 5

elif "SMD" in MODEL_NAME:
    threshold_fixed = 0.12
    th_factor = 4.1

elif "MSL" in MODEL_NAME:
    threshold_fixed = 1
    th_factor = 6.5


test_data_tmp = test_data[window_size:]
true_labels = test_labels[window_size:]

mse = np.mean(np.power(test_data_tmp - predictions, 2), axis=1)

# threshold_fixed = np.percentile(mse, 99)
print("threshold: ", threshold_fixed)

pred_y = [1 if e > threshold_fixed else 0 for e in mse]

gt, pred_adjusted = apply_adjustment(true_labels, pred_y)
print("adjusted: ", get_precision_recall_f1(gt, pred_adjusted))

pred_y, dynamic_threshold = sliding_window_anomaly_detection(mse, window_size, threshold_factor=th_factor)
gt, pred_adjusted = apply_adjustment(true_labels, pred_y)
print("adjusted with sliding: ", get_precision_recall_f1(gt, pred_adjusted))

threshold:  0.12
adjusted:  (0.6272, 0.5201, 0.5686)
adjusted with sliding:  (0.9201, 0.882, 0.9006)
